In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pulp

from scipy import stats

In [214]:
demand=pd.read_csv("demands.csv")
distance=pd.read_csv("distances.csv")
wage=pd.read_csv("salary.csv")
center_old=pd.read_csv("centers.csv")
center=pd.read_csv("centers_new.csv")

In [221]:
from sklearn.linear_model import LinearRegression
def future_demand(temp_df,forecast_x): 
    x = temp_df['Year'].to_numpy().reshape((-1, 1))
    y = temp_df['Demand'].to_numpy()
    model = LinearRegression()
    model.fit(x, y)
    y_pred = model.predict(forecast_x),0
    print(y_pred)
    
region_cols = demand['Region'].unique()
forecast_x = np.array([1, 2, 3, 4, 5]).reshape((-1, 1))

# for col in region_cols:
#     print("Region: ", col)
#     print("Forecasted Demand: ")
#     temp_df = demand[demand['Region'] == col]
#     future_demand(temp_df, forecast_x)
year_list = [1,2,3,4,5,
             1,2,3,4,5,
             1,2,3,4,5,
             1,2,3,4,5,
             1,2,3,4,5,
             1,2,3,4,5,
             1,2,3,4,5]
region_list = [1,1,1,1,1,
              2,2,2,2,2,
              3,3,3,3,3,
              4,4,4,4,4,
              5,5,5,5,5,
              6,6,6,6,6,
              7,7,7,7,7]
demand_forecast_list = [397295.66666667, 412506.86060606, 427718.05454546, 442929.24848485, 458140.44242425,
                       275250.8, 285828.2, 296405.6, 306983. , 317560.4,
                       457808.06666666, 475512.98787878, 493217.90909091, 510922.83030303, 528627.75151515,
                       406851.4, 422397.23636364, 437943.07272727, 453488.90909091, 469034.74545455,
                       417190.2, 433025.09090909, 448859.98181818, 464694.87272727, 480529.76363636,
                        508686.46666666, 528427.84242424, 548169.21818182, 567910.59393939, 587651.96969697,
                       445900.8, 462885.92727272, 479871.05454545, 496856.18181818, 513841.3090909]

demand_forecast_list = [round(item,0) for item in demand_forecast_list]
demand = pd.DataFrame(list(zip(year_list,region_list,demand_forecast_list)),
                          columns=['Period','Region','Demand']).sort_values(['Period','Region'])

demand=demand.reset_index()[['Period','Region','Demand']]

# Base One-Year Model

In [242]:
#Demand region
regions = list(demand["Region"].unique())
regionnum = len(regions)

#Period
numperiods = 6
periods = range(1, numperiods)

#New medical center
centers= list(center_old['Center'])
numcenter=len(centers)

In [243]:
# ----------- base model one year
# Variables: if a MP is opened/allocated
yvar = pulp.LpVariable.dict("y", centers, cat=pulp.LpBinary)

# Variables: amount of demand from each MP allocated to region
xvar = pulp.LpVariable.dict("x", (centers, regions), lowBound=0.0,cat=pulp.LpContinuous)


In [244]:
# ------------base model parameters
# Demand in each region in k year(dictionary) D
regiondemand = {}
for j in regions:
    regiondemand[j] = int(demand[(demand["Region"] == j)&(demand['Period']==1)]["Demand"])
        

# Medical center cost (dictionary) f
centercost = {}
for i in centers:
    centercost[i] = int(center_old[center_old['Center']==i]["Operating Costs"])
    
# Center-region distance (dictionary) d
crdistance = {}
for i in centers:
    for j in regions:
        crdistance[(i,j)] = float(distance[(distance["Center"] == i)&(distance["Region"] == j)]["Distance"])
        
# Opening cost (dictionary) c
opencost = {}
for i in centers:
    opencost[i] = int(center_old[center_old['Center']==i]["Opening Costs"])
    
    
# # Number of worker w
# workernum = {}
# for i in centers:
#     workernum[i]=int(center[center['Center']==i]["Num MPs"])
    

In [257]:
# ------------base model initialization
# Initialize model and objective sense
locationModel = pulp.LpProblem(name="LocationModel", sense=pulp.LpMinimize)

In [258]:
# -------------base model constraint

for i in range(1,5):
    locationModel += (yvar[i]==1)

for j in regions:
    locationModel += pulp.lpSum( [xvar[(i,j)] for i in centers] ) == regiondemand[j]
    
for i in centers:
    locationModel += pulp.lpSum( [xvar[(i,j)] for j in regions] ) <= 5000000*yvar[i]
 

    
# for i in centers:
#     locationModel += (pulp.lpSum( [xvar[(i,j)] for j in regions]) <=1500*wvar[i])
    


In [259]:
# ------------- base model objective function

# --- center allocation costs
obj = pulp.lpSum([ centercost[i] * yvar[i] for i in centers])

# --- center opening costs
obj += pulp.lpSum([ opencost[i] * yvar[i] for i in centers])

# --- service costs
obj += 35*pulp.lpSum( [ xvar[(i,j)] for i in centers for j in regions ] )

# --- commute cost
obj += 0.25* pulp.lpSum( [ crdistance[(i,j)] * xvar[(i,j)] for i in centers for j in regions ])


# ---government sub
obj -=40*pulp.lpSum( [ xvar[(i,j)] for i in centers for j in regions ] )



# add objective to model
locationModel += obj

In [260]:
# Write LP to file (optional, but often good to inspect model and find errors)
locationModel.writeLP("locationModel.lp")

# Solve model
locationModel.solve()
print("Status:", pulp.LpStatus[locationModel.status])

Status: Optimal


In [261]:
# Total cost
totalCost = pulp.value(locationModel.objective)
print("Total cost: " + str(totalCost))

Total cost: -9617794.16


In [232]:
for i in centers:
    if yvar[i].varValue == 1.0:
        print("Center " + str(i) + " is opened")

Center 1 is opened
Center 2 is opened
Center 3 is opened
Center 4 is opened
Center 5 is opened


In [233]:
for i in centers:
    print( str(sum([xvar[i,j].varValue for j in regions]))+ ' hours at center ' +str(i) )

783937.0 hours at center 1
445901.0 hours at center 2
0.0 hours at center 3
406851.0 hours at center 4
1272294.0 hours at center 5
0.0 hours at center 6


# Model Pre Over time

In [262]:
#Demand region
regions = list(demand["Region"].unique())
regionnum = len(regions)

#Period
numperiods = 6
periods = range(1, numperiods)

#New medical center
centers= list(center['Center'])
numcenter=len(centers)

In [263]:
### --------------Parameter
# Demand in each region in k year(dictionary) D
regiondemand = {}
for j in regions:
    for k in periods:
        regiondemand[(j,k)] = int(demand[(demand["Region"] == j)&(demand['Period']==k)]["Demand"])
        

# Medical center cost (dictionary) f
centercost = {}
for i in centers:
    centercost[i] = int(center[center['Center']==i]["Operating Costs"])
    
# Center-region distance (dictionary) d
crdistance = {}
for i in centers:
    for j in regions:
        crdistance[(i,j)] = float(distance[(distance["Center"] == i)&(distance["Region"] == j)]["Distance"])
        
# Salary s
salary={}
for k in periods:
    salary[k]=float(wage[(wage["Period"] == k)]['Salary'])
    
# OT Salary    
OTsalary={}
for k in periods:
    OTsalary[k]=float(wage[(wage["Period"] == k)]['OT Salary'])
    
# Num of worker
worker={}

for i in centers:
    worker[i]=int(center[center['Center']==i]['Num MPs'])

In [264]:
# ---------variable
# average hour per worker per year variables, 12*250=3000
xvar = pulp.LpVariable.dict("x", (centers, regions, periods), lowBound=0.0, cat=pulp.LpContinuous)

# new hire
hvar = pulp.LpVariable.dict("h", (centers,periods), lowBound=0.0, cat=pulp.LpInteger)

# worker
wvar = pulp.LpVariable.dict("w", (centers,periods), lowBound=0.0, cat=pulp.LpContinuous)


In [265]:
# Initialize model and objective sense
FlowModel = pulp.LpProblem(name="FlowModel", sense=pulp.LpMinimize)

In [266]:
# -------------  objective function

# ---- new hire cost
obj = 15000* pulp.lpSum([ hvar[(i,k)] for i in centers for k in periods ])

# --- service costs
for k in periods:
    obj += pulp.lpSum( [ xvar[(i,j,k)] * salary[k] for i in centers for j in regions ] )
    


# --- center operating costs
for k in periods:
    obj += pulp.lpSum([ centercost[i] for i in centers])

# --- commute cost
for k in periods:
    obj += 0.25* pulp.lpSum( [ (crdistance[(i,j)] * xvar[(i,j,k)]) for i in centers for j in regions])
    
# ---government sub
obj -=40*pulp.lpSum( [ (xvar[(i,j,k)] ) for i in centers for j in regions for k in periods ] )


# add objective to model
FlowModel += obj

In [267]:
#-----constraint: no overtime

# Flow worker constraint
for k in range(1,6):  
    for i in centers:
    # Worker flow
        numworkers = (wvar[(i,k-1)] if k > 1 else worker[i]) + hvar[(i,k)]
        FlowModel += (wvar[(i,k)] == numworkers)
        
# new hire constraint
for k in periods:
    for i in centers:
        FlowModel += (hvar[(i,k)]  <=100)
    
# service capacity
for k in periods:
    for i in centers:
        FlowModel += (pulp.lpSum( [xvar[(i,j,k)] for j in regions]) <=1500*wvar[(i,k)])
        


# before overtime demand constraint    
for k in periods:
    for j in regions:
        FlowModel += (pulp.lpSum( [(xvar[(i,j,k)] ) for i in centers] ) <= regiondemand[(j,k)] )
        
        

# Center capacity constraint
for k in periods:
    for i in centers:
        FlowModel += (pulp.lpSum( [(xvar[(i,j,k)]) for j in regions] ) <= 5000000)

In [268]:
# Write LP to file (optional, but often good to inspect model and find errors)
FlowModel.writeLP("FlowModel.lp")

# Solve model
FlowModel.solve()
print("Status:", pulp.LpStatus[FlowModel.status])

Status: Optimal


In [269]:
# Total cost
totalCost = pulp.value(FlowModel.objective)
print("Total cost: " + str(totalCost))

Total cost: -9096503.209999992


In [270]:
for k in periods:
    for i in centers:
        print( str( hvar[i,k].varValue) + " is hired in year "+ str(k)+ ' at center '+ str(i) )
        

100.0 is hired in year 1 at center 1
100.0 is hired in year 1 at center 2
0.0 is hired in year 1 at center 3
100.0 is hired in year 1 at center 4
100.0 is hired in year 1 at center 5
51.0 is hired in year 2 at center 1
83.0 is hired in year 2 at center 2
0.0 is hired in year 2 at center 3
61.0 is hired in year 2 at center 4
100.0 is hired in year 2 at center 5
0.0 is hired in year 3 at center 1
0.0 is hired in year 3 at center 2
0.0 is hired in year 3 at center 3
0.0 is hired in year 3 at center 4
0.0 is hired in year 3 at center 5
0.0 is hired in year 4 at center 1
0.0 is hired in year 4 at center 2
0.0 is hired in year 4 at center 3
0.0 is hired in year 4 at center 4
0.0 is hired in year 4 at center 5
0.0 is hired in year 5 at center 1
0.0 is hired in year 5 at center 2
0.0 is hired in year 5 at center 3
0.0 is hired in year 5 at center 4
0.0 is hired in year 5 at center 5


In [161]:
for k in periods:
    for i in centers:
        print( str( wvar[i,k].varValue) + " workers are working in year "+ str(k)+ ' at center '+ str(i) )

315.0 workers are working in year 1 at center 1
225.0 workers are working in year 1 at center 2
100.0 workers are working in year 1 at center 3
220.0 workers are working in year 1 at center 4
100.0 workers are working in year 1 at center 5
366.0 workers are working in year 2 at center 1
308.0 workers are working in year 2 at center 2
100.0 workers are working in year 2 at center 3
281.0 workers are working in year 2 at center 4
200.0 workers are working in year 2 at center 5
366.0 workers are working in year 3 at center 1
308.0 workers are working in year 3 at center 2
100.0 workers are working in year 3 at center 3
281.0 workers are working in year 3 at center 4
200.0 workers are working in year 3 at center 5
366.0 workers are working in year 4 at center 1
308.0 workers are working in year 4 at center 2
100.0 workers are working in year 4 at center 3
281.0 workers are working in year 4 at center 4
200.0 workers are working in year 4 at center 5
366.0 workers are working in year 5 at c

In [272]:
for k in periods:
    for i in centers:
        for j in regions:
            print( str(xvar[i,j,k].varValue)+ ' hours from center '+ str(i)+ ' to region ' +str(j) + ' in period ' + str(k))

0.0 hours from center 1 to region 1 in period 1
0.0 hours from center 1 to region 2 in period 1
0.0 hours from center 1 to region 3 in period 1
0.0 hours from center 1 to region 4 in period 1
0.0 hours from center 1 to region 5 in period 1
472500.0 hours from center 1 to region 6 in period 1
0.0 hours from center 1 to region 7 in period 1
0.0 hours from center 2 to region 1 in period 1
0.0 hours from center 2 to region 2 in period 1
0.0 hours from center 2 to region 3 in period 1
0.0 hours from center 2 to region 4 in period 1
0.0 hours from center 2 to region 5 in period 1
0.0 hours from center 2 to region 6 in period 1
337500.0 hours from center 2 to region 7 in period 1
41599.0 hours from center 3 to region 1 in period 1
0.0 hours from center 3 to region 2 in period 1
0.0 hours from center 3 to region 3 in period 1
0.0 hours from center 3 to region 4 in period 1
0.0 hours from center 3 to region 5 in period 1
0.0 hours from center 3 to region 6 in period 1
108401.0 hours from center

# Full model with Over Time

In [302]:
# ---------variable
# average hour per worker per year variables, 12*250=3000
xvar = pulp.LpVariable.dict("x", (centers, regions, periods), lowBound=0.0, cat=pulp.LpContinuous)

# new hire
hvar = pulp.LpVariable.dict("h", (centers,periods), lowBound=0.0, cat=pulp.LpInteger)

# worker
wvar = pulp.LpVariable.dict("w", (centers,periods), lowBound=0.0, cat=pulp.LpContinuous)

# overtime
ovar = pulp.LpVariable.dict("o", (centers,regions,periods), lowBound=0.0, cat=pulp.LpContinuous)

In [303]:
# Initialize model and objective sense
FlowModel1 = pulp.LpProblem(name="FlowModel", sense=pulp.LpMinimize)

In [304]:
# -------------  objective function

# ---- new hire cost
obj = 15000* pulp.lpSum([ hvar[(i,k)] for i in centers for k in periods ])

# --- service costs
for k in periods:
    obj += pulp.lpSum( [ xvar[(i,j,k)] * salary[k] for i in centers for j in regions ] )
    
# OT cost
for k in periods:
    obj += pulp.lpSum( [ ovar[(i,j,k)] * OTsalary[k] for i in centers for j in regions] )

# --- center operating costs
for k in periods:
    obj += pulp.lpSum([ centercost[i] for i in centers])

# --- commute cost
for k in periods:
    obj += 0.25* pulp.lpSum( [ (crdistance[(i,j)] * (xvar[(i,j,k)]+ovar[(i,j,k)])) for i in centers for j in regions])
    
# ---government sub
obj -=40*pulp.lpSum( [ (xvar[(i,j,k)] ) for i in centers for j in regions for k in periods ] )
    
    
# ---government sub OT
obj -=40*pulp.lpSum( [ (ovar[(i,j,k)] ) for i in centers for j in regions for k in periods ] )



# add objective to model
FlowModel1 += obj

In [312]:
#-----constraint: have overtime

# Flow worker constraint
for k in range(1,6):  
    for i in centers:
    # Worker flow
        numworkers = (wvar[(i,k-1)] if k > 1 else worker[i]) + hvar[(i,k)]
        FlowModel1 += (wvar[(i,k)] == numworkers)
        
# new hire constraint
for k in periods:
    for i in centers:
        FlowModel1 += (hvar[(i,k)]  <=100)
    
# service capacity
for k in periods:
    for i in centers:
        FlowModel1 += (pulp.lpSum( [xvar[(i,j,k)] for j in regions]) <=1500*wvar[(i,k)])
        

# overtime assumption: no more than 6 hours overtime per day -> 12*250=3000        
for k in periods:
    for i in centers:
        FlowModel1 += (pulp.lpSum( [ovar[(i,j,k)] for j in regions] ) <=1500*wvar[(i,k)])
    
# before overtime demand constraint    
for k in periods:
    for j in regions:
        FlowModel1 += (pulp.lpSum( [(xvar[(i,j,k)] ) for i in centers] ) <= regiondemand[(j,k)] )
        
# after overtime constraint        
for k in periods:
    for j in regions:
        FlowModel1 += (pulp.lpSum( [(xvar[(i,j,k)] + ovar[(i,j,k)]) for i in centers] ) == regiondemand[(j,k)] )
        
        

# Center capacity constraint
for k in periods:
    for i in centers:
        FlowModel1 += (pulp.lpSum( [(xvar[(i,j,k)] + ovar[(i,j,k)]) for j in regions] ) <= 5000000)

In [313]:
# Write LP to file (optional, but often good to inspect model and find errors)
FlowModel1.writeLP("FlowModel1.lp")

# Solve model
FlowModel1.solve()
print("Status:", pulp.LpStatus[FlowModel.status])

Status: Optimal


In [314]:
# Total cost
totalCost = pulp.value(FlowModel1.objective)
print("Total cost: " + str(totalCost))

Total cost: 46024180.67000001


In [319]:
for k in periods:
    for i in centers:
        print( str( hvar[i,k].varValue) + " is hired in year "+ str(k)+ ' at center '+ str(i) )
        

100.0 is hired in year 1 at center 1
100.0 is hired in year 1 at center 2
100.0 is hired in year 1 at center 3
100.0 is hired in year 1 at center 4
100.0 is hired in year 1 at center 5
100.0 is hired in year 2 at center 1
100.0 is hired in year 2 at center 2
100.0 is hired in year 2 at center 3
100.0 is hired in year 2 at center 4
100.0 is hired in year 2 at center 5
100.0 is hired in year 3 at center 1
100.0 is hired in year 3 at center 2
100.0 is hired in year 3 at center 3
100.0 is hired in year 3 at center 4
100.0 is hired in year 3 at center 5
77.0 is hired in year 4 at center 1
0.0 is hired in year 4 at center 2
0.0 is hired in year 4 at center 3
0.0 is hired in year 4 at center 4
100.0 is hired in year 4 at center 5
0.0 is hired in year 5 at center 1
0.0 is hired in year 5 at center 2
0.0 is hired in year 5 at center 3
0.0 is hired in year 5 at center 4
0.0 is hired in year 5 at center 5


In [320]:
for k in periods:
    for i in centers:
        print( str( wvar[i,k].varValue) + " workers are working in year "+ str(k)+ ' at center '+ str(i) )

315.0 workers are working in year 1 at center 1
225.0 workers are working in year 1 at center 2
200.0 workers are working in year 1 at center 3
220.0 workers are working in year 1 at center 4
100.0 workers are working in year 1 at center 5
415.0 workers are working in year 2 at center 1
325.0 workers are working in year 2 at center 2
300.0 workers are working in year 2 at center 3
320.0 workers are working in year 2 at center 4
200.0 workers are working in year 2 at center 5
515.0 workers are working in year 3 at center 1
425.0 workers are working in year 3 at center 2
400.0 workers are working in year 3 at center 3
420.0 workers are working in year 3 at center 4
300.0 workers are working in year 3 at center 5
592.0 workers are working in year 4 at center 1
425.0 workers are working in year 4 at center 2
400.0 workers are working in year 4 at center 3
420.0 workers are working in year 4 at center 4
400.0 workers are working in year 4 at center 5
592.0 workers are working in year 5 at c

In [321]:
for k in periods:
    for i in centers:
        for j in regions:
            print( str( ovar[i,j,k].varValue) + " hours of overtime in year "+ str(k)+ ' at center '+ str(i) +' to '+ str(j) )

0.0 hours of overtime in year 1 at center 1 to 1
0.0 hours of overtime in year 1 at center 1 to 2
43873.0 hours of overtime in year 1 at center 1 to 3
0.0 hours of overtime in year 1 at center 1 to 4
117190.0 hours of overtime in year 1 at center 1 to 5
311437.0 hours of overtime in year 1 at center 1 to 6
0.0 hours of overtime in year 1 at center 1 to 7
0.0 hours of overtime in year 1 at center 2 to 1
0.0 hours of overtime in year 1 at center 2 to 2
211231.0 hours of overtime in year 1 at center 2 to 3
0.0 hours of overtime in year 1 at center 2 to 4
0.0 hours of overtime in year 1 at center 2 to 5
0.0 hours of overtime in year 1 at center 2 to 6
108401.0 hours of overtime in year 1 at center 2 to 7
97296.0 hours of overtime in year 1 at center 3 to 1
0.0 hours of overtime in year 1 at center 3 to 2
202704.0 hours of overtime in year 1 at center 3 to 3
0.0 hours of overtime in year 1 at center 3 to 4
0.0 hours of overtime in year 1 at center 3 to 5
0.0 hours of overtime in year 1 at c

In [322]:
for k in periods:
    for i in centers:
        for j in regions:
            print( str(xvar[i,j,k].varValue + ovar[i,j,k].varValue)+ ' hours from center '+ str(i)+ ' to region ' +str(j) + ' in period ' + str(k))

0.0 hours from center 1 to region 1 in period 1
275251.0 hours from center 1 to region 2 in period 1
43873.0 hours from center 1 to region 3 in period 1
0.0 hours from center 1 to region 4 in period 1
117190.0 hours from center 1 to region 5 in period 1
508686.0 hours from center 1 to region 6 in period 1
0.0 hours from center 1 to region 7 in period 1
0.0 hours from center 2 to region 1 in period 1
0.0 hours from center 2 to region 2 in period 1
211231.0 hours from center 2 to region 3 in period 1
0.0 hours from center 2 to region 4 in period 1
0.0 hours from center 2 to region 5 in period 1
0.0 hours from center 2 to region 6 in period 1
445901.0 hours from center 2 to region 7 in period 1
397296.0 hours from center 3 to region 1 in period 1
0.0 hours from center 3 to region 2 in period 1
202704.0 hours from center 3 to region 3 in period 1
0.0 hours from center 3 to region 4 in period 1
0.0 hours from center 3 to region 5 in period 1
0.0 hours from center 3 to region 6 in period 1
0